In [1]:
####################################################

from __future__ import division

import math
import nltk
import scipy  # https://docs.scipy.org/doc/scipy/reference/spatial.distance.html
              # https://machinelearning1.wordpress.com/2013/04/10/calculating-spatial-distance-metric-in-python/
import gensim
import decimal
import collections
import numpy as np
import pandas as pd
import xgboost as xgb

####################################################

Using TensorFlow backend.
/opt/anaconda3/envs/python2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

df_train = pd.read_pickle("./data/df_train_s.pkl")
df_test = pd.read_pickle("./data/df_test.pkl")

In [3]:
# df_ = df_train[['id', 'sbj_share', 'verb_share','obj_share', 'svo_cnt1', 'svo_cnt2', 'doc_sim' ]]
# df_.to_pickle("./data/df_train_s.pkl")

In [4]:
df_train.sample(n=10)

,id,sbj_share,verb_share,obj_share,svo_cnt1,svo_cnt2,doc_sim
248200,248200,0.0,0.0,0.0,0,0,0.911897
353591,353591,0.0,0.0,0.0,0,0,0.997023
229626,229626,0.0,0.0,0.0,0,0,0.897367
170156,170156,1.0,1.0,1.0,2,2,0.955990
169884,169884,0.0,0.0,0.0,1,0,0.988462
318784,318784,0.0,0.0,0.0,0,0,0.709531
242464,242464,1.0,1.0,1.0,1,1,0.979788
21391,21391,1.0,0.0,1.0,1,1,0.951001
352365,352365,0.0,0.0,1.0,1,1,0.935561
307519,307519,0.0,0.0,0.0,1,0,0.936812


In [5]:
def svo_share(row, train_data = True ):
    if train_data:
#         row_index = row.id
        row_index = df_train[df_train.id == row.id].index[0]
    
        svo_cnt1 = df_train['svo_cnt1'][row_index]
        svo_cnt2 = df_train['svo_cnt2'][row_index]
        sbj_share = df_train['sbj_share'][row_index]
        verb_share = df_train['verb_share'][row_index]
        obj_share = df_train['obj_share'][row_index]
    else:
#         row_index = row.test_id 
        row_index = df_test[df_test.test_id == row.test_id].index[0]
        
        svo_cnt1 = df_test['svo_cnt1'][row_index]
        svo_cnt2 = df_test['svo_cnt2'][row_index]
        sbj_share = df_test['obj_share'][row_index]
        verb_share = df_test['verb_share'][row_index]
        obj_share = df_test['obj_share'][row_index]
    return svo_cnt1, svo_cnt2, sbj_share, verb_share, obj_share
        

In [6]:
# df_ = training_data

# df_['svo_cnt1'], df_['svo_cnt2'], \
# df_['sbj_share'], df_['verb_share'], df_['obj_share']\
# = zip( *df_.apply(lambda row: svo_share(row , train_data = True), axis=1))


In [7]:
# df_t = testing_data

# df_t['svo_cnt1'], df_t['svo_cnt2'], \
# df_t['sbj_share'], df_t['verb_share'], df_t['obj_share']\
# = zip( *df_t.apply(lambda row: svo_share(row , train_data = False), axis=1))

In [8]:
# df_.head()


In [9]:
# df_t.head()

In [10]:
stopwords = set(nltk.corpus.stopwords.words('english'))

####################################################

# convert sentences to vectors

def sent2vec( sentence ):
    
    words             = str( sentence ).lower().decode('utf-8')
    words             = nltk.word_tokenize( words )
    words             = [word for word in words if not word in stopwords]
    words             = [word for word in words if word.isalpha()]
    
    word_vector_list  = []

    for word in words:
        
        try:
            
            word_vector_list.append( word2vec[word] )
            
        except:
            
            continue
    
    
    word_vector_array = np.array( word_vector_list )
    
    if len(word_vector_list) == 0:
        sentence_vector = np.zeros(300)
    else:
        sentence_vector = word_vector_array.sum( axis=0 )
    
    return sentence_vector

####################################################

In [11]:
####################################################

def braycurtis_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.braycurtis (
                                              np.nan_to_num( question1_vector ), 
                                              np.nan_to_num( question2_vector )
                                             )

####################################################

In [12]:
####################################################

def canberra_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.canberra (
                                            np.nan_to_num( question1_vector ), 
                                            np.nan_to_num( question2_vector )
                                           )

####################################################

In [13]:
####################################################

def chebyshev_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.chebyshev (
                                             np.nan_to_num( question1_vector ), 
                                             np.nan_to_num( question2_vector )
                                            )

####################################################

In [14]:
####################################################

def cityblock_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.cityblock (
                                             np.nan_to_num( question1_vector ), 
                                             np.nan_to_num( question2_vector )
                                            )

####################################################

In [15]:
####################################################

def correlation_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.correlation (
                                               np.nan_to_num( question1_vector ), 
                                               np.nan_to_num( question2_vector )
                                              )

####################################################

In [16]:
####################################################

def cosine_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.cosine (
                                          np.nan_to_num( question1_vector ), 
                                          np.nan_to_num( question2_vector )
                                         )

####################################################

In [17]:
####################################################

def euclidean_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.euclidean (
                                             np.nan_to_num( question1_vector ), 
                                             np.nan_to_num( question2_vector )
                                            )

####################################################

In [18]:
####################################################

def hamming_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.hamming (
                                           np.nan_to_num( question1_vector ), 
                                           np.nan_to_num( question2_vector )
                                          )

####################################################

In [19]:
####################################################

def jaccard_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.jaccard (
                                           np.nan_to_num( question1_vector ), 
                                           np.nan_to_num( question2_vector )
                                          )

####################################################

In [20]:
####################################################

def matching_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.matching (
                                            np.nan_to_num( question1_vector ), 
                                            np.nan_to_num( question2_vector )
                                           )

####################################################

In [21]:
####################################################

def minkowski_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.minkowski (
                                             np.nan_to_num( question1_vector ), 
                                             np.nan_to_num( question2_vector ),
                                             3
                                            )

####################################################

In [22]:
####################################################

def russellrao_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.russellrao (
                                              np.nan_to_num( question1_vector ), 
                                              np.nan_to_num( question2_vector )
                                             )

####################################################

In [23]:
####################################################

def sqeuclidean_distance ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.spatial.distance.sqeuclidean (
                                               np.nan_to_num( question1_vector ), 
                                               np.nan_to_num( question2_vector )
                                              )

####################################################

In [24]:
stopwords = set(nltk.corpus.stopwords.words('english'))

####################################################

def wordmovers_distance ( row ):

    question1 = str(row['question1']).lower().split()
    question2 = str(row['question2']).lower().split()
    
    question1 = [word for word in question1 if word not in stopwords]
    question2 = [word for word in question2 if word not in stopwords]

    return word2vec.wmdistance(question1, question2)

####################################################

In [25]:
stopwords = set(nltk.corpus.stopwords.words('english'))

####################################################

def wordmovers_normalized_distance ( row ):

    question1 = str(row['question1']).lower().split()
    question2 = str(row['question2']).lower().split()
    
    question1 = [word for word in question1 if word not in stopwords]
    question2 = [word for word in question2 if word not in stopwords]

    return word2vec_normalized.wmdistance(question1, question2)

####################################################

In [26]:
####################################################

def question1_skew ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    
    return scipy.stats.skew(np.nan_to_num( question1_vector ))

####################################################

In [27]:
####################################################

def question2_skew ( row ):

    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.stats.skew(np.nan_to_num( question2_vector ))

####################################################

In [28]:
####################################################

def question1_kurtosis ( row ):

    question1_vector = question1_vector_lookup[row['question1']]
    
    return scipy.stats.kurtosis(np.nan_to_num( question1_vector ))

####################################################

In [29]:
####################################################

def question2_kurtosis ( row ):

    question2_vector = question2_vector_lookup[row['question2']]
    
    return scipy.stats.kurtosis(np.nan_to_num( question2_vector ))

####################################################

In [30]:
from fuzzywuzzy import fuzz

####################################################

def fuzzy_qratio ( row ):
    return fuzz.QRatio(str(row['question1']), str(row['question2']))

####################################################

In [31]:
from fuzzywuzzy import fuzz

####################################################

def fuzzy_WRatio ( row ):
    return fuzz.WRatio(str(row['question1']), str(row['question2']))

####################################################

In [32]:
from fuzzywuzzy import fuzz

####################################################

def fuzzy_partial_ratio ( row ):
    return fuzz.partial_ratio(str(row['question1']), str(row['question2']))

####################################################

In [33]:
from fuzzywuzzy import fuzz

####################################################

def fuzzy_partial_token_set_ratio ( row ):
    return fuzz.partial_token_set_ratio(str(row['question1']), str(row['question2']))

####################################################

In [34]:
from fuzzywuzzy import fuzz

####################################################

def fuzzy_partial_token_sort_ratio ( row ):
    return fuzz.partial_token_sort_ratio(str(row['question1']), str(row['question2']))

####################################################

In [35]:
from fuzzywuzzy import fuzz

####################################################

def fuzzy_token_set_ratio ( row ):
    return fuzz.token_set_ratio(str(row['question1']), str(row['question2']))

####################################################

In [36]:
from fuzzywuzzy import fuzz

####################################################

def fuzzy_token_sort_ratio ( row ):
    return fuzz.token_sort_ratio(str(row['question1']), str(row['question2']))

####################################################


In [37]:
stops = set(nltk.corpus.stopwords.words('english'))

####################################################

# determine the number of matching words between question1 and question2 using a simple count and normalize

def word_match_simple_count ( row ):
    
    question1_words = {}
    question2_words = {}
    
    for word in str( row['question1'] ).lower().split():
        
        if word not in stops:
            
            question1_words[word] = 1
            
    for word in str( row['question2'] ).lower().split():
        
        if word not in stops:
            
            question2_words[word] = 1
            
    if len(question1_words) == 0 or len(question2_words) == 0:
        return 0

    shared_words_in_question1 = [ word for word in question1_words.keys() if word in question2_words ]
    shared_words_in_question2 = [ word for word in question2_words.keys() if word in question1_words ]
    
    return ( len(shared_words_in_question1) + len(shared_words_in_question2) ) / \
           ( len(question1_words)           + len(question2_words)           )

####################################################

In [38]:
####################################################

# calculate a weight for each word

# If a word frequency is below the minimum count, we ignore the word
# smoothing reduces the impact of rare words

def get_word_weight ( count, smoothing, minimum_count ):

    if count < minimum_count:

        return 0
    
    else:

        return 1 / (count + smoothing)

####################################################

In [39]:
stops = set(nltk.corpus.stopwords.words("english"))

####################################################

# determine the number of matching words between question1 and question2 using a per word weight and normalize

def word_match_simple_weight ( row ):
    
    question1_words = {}
    question2_words = {}
    
    for word in str( row['question1'] ).lower().split():
        
        if word not in stops:
            
            question1_words[word] = 1
            
    for word in str( row['question2'] ).lower().split():
        
        if word not in stops:
            
            question2_words[word] = 1
            
    if len(question1_words) == 0 or len(question2_words) == 0:
        return 0
    
    shared_weights = [ word_weights.get(word, 0) for word in question1_words.keys() if word in question2_words ] + \
                     [ word_weights.get(word, 0) for word in question2_words.keys() if word in question1_words ]
        
    total_weights  = [ word_weights.get(word, 0) for word in question1_words ] + \
                     [ word_weights.get(word, 0) for word in question2_words ]
    
    return np.sum( shared_weights ) / np.sum( total_weights )

####################################################

In [40]:
print 'Load training and testing dataset'

training_data       = pd.read_csv('./mini_train.csv' )
testing_data        = pd.read_csv('./mini_test.csv' )

Load training and testing dataset


In [ ]:
####################################################



word2vec            = gensim.models.KeyedVectors.load_word2vec_format (
                                                            './wordvec/GoogleNews-vectors-negative300.bin', 
                                                                    binary = True
                                                                      )

word2vec_normalized = gensim.models.KeyedVectors.load_word2vec_format (
                                                            './wordvec//GoogleNews-vectors-negative300.bin', 
                                                                    binary = True
                                                                      )
word2vec_normalized.init_sims(replace=True)

training_questions  = pd.Series ( 
                                 training_data['question1'].tolist() +
                                 training_data['question2'].tolist() 
                                ).astype(str)

testing_questions   = pd.Series ( 
                                 testing_data['question1'].tolist() +
                                 testing_data['question2'].tolist() 
                                ).astype(str)

question1_vector_lookup = {}
question2_vector_lookup = {}

count = 0
for index, row in training_data.iterrows():
    question1_vector_lookup[row['question1']] = sent2vec(row['question1'])
    question2_vector_lookup[row['question2']] = sent2vec(row['question2'])
    count += 1
    if count % 100000 == 0:
        print 'training: ' + str(count)

count = 0
for index, row in testing_data.iterrows():
    question1_vector_lookup[row['question1']] = sent2vec(row['question1'])
    question2_vector_lookup[row['question2']] = sent2vec(row['question2'])
    count += 1
    if count % 100000 == 0:
        print 'testing: ' + str(count)

####################################################

print 'Calculate a weight for each word from the training and testing datasets'

word_count         = collections.defaultdict(int)

for question in training_questions:
    for word in question.lower().split():
        word_count[word] += 1

for question in testing_questions:
    for word in question.lower().split():
        word_count[word] += 1

word_weights        = {word : get_word_weight ( 
                                               count, 
                                               smoothing     = 10000, 
                                               minimum_count = 2
                                              ) for word, count in word_count.items()}

####################################################

In [ ]:
from sklearn.cross_validation import train_test_split

####################################################

print 'Prepare training and testing data'

x_train                             = pd.DataFrame()
x_test                              = pd.DataFrame()

####################################################


x_train['svo_cnt1'], x_train['svo_cnt2'], \
x_train['sbj_share'], x_train['verb_share'], x_train['obj_share']\
= zip( *training_data.apply(lambda row: svo_share(row , train_data = True), axis=1))

x_test['svo_cnt1'], x_test['svo_cnt2'], \
x_test['sbj_share'], x_test['verb_share'], x_test['obj_share']\
= zip( *testing_data.apply(lambda row: svo_share(row , train_data = False), axis=1))

In [ ]:



x_train['word_match_simple_count']  = training_data.apply (
                                                           func = word_match_simple_count, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - word_match_simple_count'

x_train['word_match_simple_weight'] = training_data.apply (
                                                           func = word_match_simple_weight, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - word_match_simple_weight'

x_train['braycurtis_distance']      = training_data.apply (
                                                           func = braycurtis_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - braycurtis_distance'

x_train['canberra_distance']        = training_data.apply (
                                                           func = canberra_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - canberra_distance'

x_train['chebyshev_distance']       = training_data.apply (
                                                           func = chebyshev_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - chebyshev_distance'

x_train['cityblock_distance']       = training_data.apply (
                                                           func = cityblock_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - cityblock_distance'

x_train['correlation_distance']     = training_data.apply (
                                                           func = correlation_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - correlation_distance'

x_train['cosine_distance']          = training_data.apply (
                                                           func = cosine_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - cosine_distance'

x_train['euclidean_distance']       = training_data.apply (
                                                           func = euclidean_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - euclidean_distance'

x_train['hamming_distance']         = training_data.apply (
                                                           func = hamming_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - hamming_distance'

x_train['jaccard_distance']         = training_data.apply (
                                                           func = jaccard_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - jaccard_distance'

x_train['matching_distance']        = training_data.apply (
                                                           func = matching_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - matching_distance'

x_train['minkowski_distance']       = training_data.apply (
                                                           func = minkowski_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - minkowski_distance'

x_train['russellrao_distance']      = training_data.apply (
                                                           func = russellrao_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - russellrao_distance'

x_train['sqeuclidean_distance']     = training_data.apply (
                                                           func = sqeuclidean_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - sqeuclidean_distance'

x_train['wordmovers_distance']     = training_data.apply (
                                                           func = wordmovers_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - wordmovers_distance'

x_train['wordmovers_normalized_distance'] = training_data.apply (
                                                           func = wordmovers_normalized_distance, 
                                                           axis = 1, 
                                                           raw  = True
                                                          )
print 'train - wordmovers_normalized_distance'

x_train['length_question1']               = training_data.question1.apply(lambda x: len(str(x)))
x_train['length_question2']               = training_data.question2.apply(lambda x: len(str(x)))
x_train['length_difference']              = x_train['length_question1'] - x_train['length_question2']
x_train['number_characters_question1']    = training_data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
x_train['number_characters_question2']    = training_data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
x_train['number_words_question1']         = training_data.question1.apply(lambda x: len(str(x).split()))
x_train['number_words_question2']         = training_data.question2.apply(lambda x: len(str(x).split()))
x_train['common_words']                   = training_data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
print 'train - basic features'

x_train['skew_question1_vector']          = training_data.apply (
                                                                 func = question1_skew,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - skew_question1_vector'

x_train['skew_question2_vector']          = training_data.apply (
                                                                 func = question2_skew,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - skew_question2_vector'

x_train['kurtosis_question1_vector']      = training_data.apply (
                                                                 func = question1_kurtosis,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - kurtosis_question1_vector'

x_train['kurtosis_question2_vector']      = training_data.apply (
                                                                 func = question2_kurtosis,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - kurtosis_question2_vector'

x_train['fuzzy_qratio']                   = training_data.apply (
                                                                 func = fuzzy_qratio,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - fuzzy_qratio'

x_train['fuzzy_WRatio']                   = training_data.apply (
                                                                 func = fuzzy_WRatio,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - fuzzy_WRatio'

x_train['fuzzy_partial_ratio']            = training_data.apply (
                                                                 func = fuzzy_partial_ratio,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - fuzzy_partial_ratio'

x_train['fuzzy_partial_token_set_ratio']  = training_data.apply (
                                                                 func = fuzzy_partial_token_set_ratio,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - fuzzy_partial_token_set_ratio'

x_train['fuzzy_partial_token_sort_ratio'] = training_data.apply (
                                                                 func = fuzzy_partial_token_sort_ratio,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - fuzzy_partial_token_sort_ratio'

x_train['fuzzy_token_set_ratio']          = training_data.apply (
                                                                 func = fuzzy_token_set_ratio,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - fuzzy_token_set_ratio'

x_train['fuzzy_token_sort_ratio']         = training_data.apply (
                                                                 func = fuzzy_token_sort_ratio,
                                                                 axis = 1, 
                                                                 raw  = True
                                                                )
print 'train - fuzzy_token_sort_ratio'

####################################################

x_test['word_match_simple_count']  = testing_data.apply (
                                                         func = word_match_simple_count, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - word_match_simple_count'

x_test['word_match_simple_weight'] = testing_data.apply (
                                                         func = word_match_simple_weight, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - word_match_simple_weight'

x_test['braycurtis_distance']      = testing_data.apply (
                                                         func = braycurtis_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - braycurtis_distance'

x_test['canberra_distance']        = testing_data.apply (
                                                         func = canberra_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - canberra_distance'

x_test['chebyshev_distance']       = testing_data.apply (
                                                         func = chebyshev_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - chebyshev_distance'

x_test['cityblock_distance']       = testing_data.apply (
                                                         func = cityblock_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - cityblock_distance'

x_test['correlation_distance']     = testing_data.apply (
                                                         func = correlation_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - correlation_distance'

x_test['cosine_distance']          = testing_data.apply (
                                                         func = cosine_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - cosine_distance'

x_test['euclidean_distance']       = testing_data.apply (
                                                         func = euclidean_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - euclidean_distance'

x_test['hamming_distance']         = testing_data.apply (
                                                         func = hamming_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - hamming_distance'

x_test['jaccard_distance']         = testing_data.apply (
                                                         func = jaccard_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - jaccard_distance'

x_test['matching_distance']        = testing_data.apply (
                                                         func = matching_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - matching_distance'

x_test['minkowski_distance']       = testing_data.apply (
                                                         func = minkowski_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - minkowski_distance'

x_test['russellrao_distance']      = testing_data.apply (
                                                         func = russellrao_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - russellrao_distance'

x_test['sqeuclidean_distance']     = testing_data.apply (
                                                         func = sqeuclidean_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - sqeuclidean_distance'

x_test['wordmovers_distance']      = testing_data.apply (
                                                         func = wordmovers_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - wordmovers_distance'

x_test['wordmovers_normalized_distance'] = testing_data.apply (
                                                         func = wordmovers_normalized_distance, 
                                                         axis = 1, 
                                                         raw  = True
                                                        )
print 'test - wordmovers_normalized_distance'

x_test['length_question1']               = testing_data.question1.apply(lambda x: len(str(x)))
x_test['length_question2']               = testing_data.question2.apply(lambda x: len(str(x)))
x_test['length_difference']              = x_test['length_question1'] - x_train['length_question2']
x_test['number_characters_question1']    = testing_data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
x_test['number_characters_question2']    = testing_data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
x_test['number_words_question1']         = testing_data.question1.apply(lambda x: len(str(x).split()))
x_test['number_words_question2']         = testing_data.question2.apply(lambda x: len(str(x).split()))
x_test['common_words']                   = testing_data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
print 'test - basic features'

x_test['skew_question1_vector']          = testing_data.apply (
                                                               func = question1_skew,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - skew_question1_vector'

x_test['skew_question2_vector']          = testing_data.apply (
                                                               func = question2_skew,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - skew_question2_vector'

x_test['kurtosis_question1_vector']      = testing_data.apply (
                                                               func = question1_kurtosis,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - kurtosis_question1_vector'

x_test['kurtosis_question2_vector']      = testing_data.apply (
                                                               func = question2_kurtosis,
                                                               axis = 1, 
                                                               raw  = True
                                                              )

print 'test - kurtosis_question2_vector'

x_test['fuzzy_qratio']                   = testing_data.apply (
                                                               func = fuzzy_qratio,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - fuzzy_qratio'

x_test['fuzzy_WRatio']                   = testing_data.apply (
                                                               func = fuzzy_WRatio,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - fuzzy_WRatio'

x_test['fuzzy_partial_ratio']            = testing_data.apply (
                                                               func = fuzzy_partial_ratio,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - fuzzy_partial_ratio'

x_test['fuzzy_partial_token_set_ratio']  = testing_data.apply (
                                                               func = fuzzy_partial_token_set_ratio,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - fuzzy_partial_token_set_ratio'

x_test['fuzzy_partial_token_sort_ratio'] = testing_data.apply (
                                                               func = fuzzy_partial_token_sort_ratio,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - fuzzy_partial_token_sort_ratio'

x_test['fuzzy_token_set_ratio']          = testing_data.apply (
                                                               func = fuzzy_token_set_ratio,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - fuzzy_token_set_ratio'

x_test['fuzzy_token_sort_ratio']         = testing_data.apply (
                                                               func = fuzzy_token_sort_ratio,
                                                               axis = 1, 
                                                               raw  = True
                                                              )
print 'test - fuzzy_token_sort_ratio'

####################################################

y_train                             = training_data['is_duplicate'].values

####################################################

print 'Split the data for training'

x_train, x_valid, y_train, y_valid  = train_test_split (
                                                        x_train,
                                                        y_train, 
                                                        test_size    = 0.2, 
                                                        random_state = 4242
                                                       )

####################################################

print 'Convert data to XGB format'

data_train                          = xgb.DMatrix (
                                                   data  = x_train,  
                                                   label = y_train
                                                  )

data_validate                       = xgb.DMatrix (
                                                   data  = x_valid, 
                                                   label = y_valid
                                                  )

####################################################

data_test                           = xgb.DMatrix (
                                                   data  = x_test
                                                  )

####################################################

In [ ]:
print x_test.head(5)

In [ ]:
####################################################

print 'Execute the XGBoost model'

XGB_parameters                                 = {}
XGB_parameters['objective']                    = 'binary:logistic'
XGB_parameters['eval_metric']                  = 'logloss'
XGB_parameters['eta']                          = 0.02
XGB_parameters['max_depth']                    = 4

XGB_watchlist                                  = [
                                                  (data_train,      'train'), 
                                                  (data_validate,   'valid')
                                                 ]

XGB_booster = xgb.train (
                         params                = XGB_parameters, 
                         dtrain                = data_train, 
                         num_boost_round       = 1000, 
                         evals                 = XGB_watchlist, 
                         early_stopping_rounds = 50, 
                         verbose_eval          = 10
                        )

####################################################

In [ ]:
####################################################

print 'Make predictions and create submission file'

predictions                    = XGB_booster.predict( data_test )

submission                     = pd.DataFrame()

submission['test_id']          = testing_data['test_id']
submission['is_duplicate']     = predictions

submission.to_csv (
                   path_or_buf = 'XGBOOST_submission_to_kaggle.csv', 
                   index       = False
                  )

####################################################